In [1]:
import json
import os
from pyspark.ml import Pipeline

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

#spark = sparknlp.start(spark32=True) #If we are using spark = 3.2.0 and above
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

train_CoNLL_file = 'data/vwb_converted_dataset.conll'
test_CoNLL_file = 'data/vwb_test_dataset_file.conll'
CSV_file = 'UserStory_For_Testing.csv'
epochs = 100

spark

Spark NLP version:  3.3.4
Apache Spark version:  3.1.2


In [2]:
#userStory = str(input())

In [3]:
with open(CSV_file) as f:
  print (f.read())

#with open('eng.testa') as f:
  c = f.read()
#userstory=userStory.split('.')
#print(userstory)
#c = userStory.r

print(c[:1000])

Userstory,Timeline,Goal,Category,Measure
"Show me the revenue across different region, different verticals and across different deparment.",,,"Vertical, region, department",Revenue
Show me revenue for different accounts. Trend of revenue for last 5 years and also the total cost by region.,5 years,,"Accounts, region","Revenue, total cost"
"I would like to visualize profit achieved last year and also the loss incurred, What is the targeted revenue and profit for the next year.","Last Year, Next year",Targeted Revenue,,"Profit, Loss"
"What Is the sales target for next year, Show the number of employees in each department. Also show department wise net profit",Next year,Sales Target,"Employees, Department",Net profit
Plot the number of products sold in a year. Plot number of products in each category. Also the revenue generated across category.,Year,,Category,"Number of product, revenue"
Show me the total premium earned last year and also the average claims incurred per policy type,Last ye

In [4]:
with open(train_CoNLL_file) as f:
  c = f.read()

print(c[:40000])

-DOCSTART- -X- -X- O 

Show VB VB O
me PRP PRP O
Rate NNP NNP B-Msr
Chg NNP NNP I-Msr
% NN NN I-Msr
for IN IN O
all DT DT O
portfolio NN NN B-Cat

What WP WP O
Is VBZ VBZ O
the DT DT O
sales NNS NNS B-Goal
target NN NN I-Goal
for IN IN O
next JJ JJ B-Tmln
year NN NN I-Tmln
, , , O
Show NNP NNP O
the DT DT O
number NN NN O
of IN IN O
employees NNS NNS B-Cat
in IN IN O
each DT DT O
department NN NN B-Cat
. . . O
Also RB RB O
show NN NN O
department NN NN B-Cat
wise JJ JJ O
net NN NN B-Msr
profit NN NN I-Msr


Show VB VB O
me PRP PRP O
Retention NN NN B-Msr
% NN NN I-Msr
for IN IN O
all DT DT O
client NN NN B-Cat

Show VB VB O
me PRP PRP O
Bound NNP NNP B-Msr
Premium NNP NNP I-Msr
for IN IN O
all DT DT O
risk NN NN B-Cat
group NN NN I-Cat

Show VB VB O
me PRP PRP O
Overheads NNS NNS B-Msr
for IN IN O
all DT DT O
broker NN NN B-Cat

Show VB VB O
me PRP PRP O
EBITDA NN NN B-Msr
for IN IN O
all DT DT O
lloyd NNP NNP B-Cat
's POS POS I-Cat
broker NN NN I-Cat

Show VB VB O
me PRP PRP O
Revenue

In [5]:
from sparknlp.training import CoNLL
training_data  = CoNLL().readDataset(spark, train_CoNLL_file)
training_data .show(truncate = False)

+-------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
training_data.select('sentence').take(100)

[Row(sentence=[Row(annotatorType='document', begin=0, end=35, result='Show me Rate Chg % for all portfolio', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', begin=0, end=126, result='What Is the sales target for next year , Show the number of employees in each department . Also show department wise net profit', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', begin=0, end=33, result='Show me Retention % for all client', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', begin=0, end=39, result='Show me Bound Premium for all risk group', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', begin=0, end=31, result='Show me Overheads for all broker', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentence=[Row(annotatorType='document', begin=0, end=37, result="Show me EBITDA for all lloyd 's broker", metadata={'sentence': '0

In [7]:
training_data.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tr

In [8]:
training_data.cache()

DataFrame[text: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentence: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, token: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, pos: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, label: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>]

In [9]:
%time training_data.count()

Wall time: 1.18 s


2867

In [10]:
import pyspark.sql.functions as F

training_data.select(F.explode(F.arrays_zip(training_data.token.result, 
                                            training_data.pos.result, 
                                            training_data.label.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("token"),
                     F.expr("cols['1']").alias("pos"),
                     F.expr("cols['2']").alias("ner_label")).show(truncate=False)

+---------+---+---------+
|token    |pos|ner_label|
+---------+---+---------+
|Show     |VB |O        |
|me       |PRP|O        |
|Rate     |NNP|B-Msr    |
|Chg      |NNP|I-Msr    |
|%        |NN |I-Msr    |
|for      |IN |O        |
|all      |DT |O        |
|portfolio|NN |B-Cat    |
|What     |WP |O        |
|Is       |VBZ|O        |
|the      |DT |O        |
|sales    |NNS|B-Goal   |
|target   |NN |I-Goal   |
|for      |IN |O        |
|next     |JJ |B-Tmln   |
|year     |NN |I-Tmln   |
|,        |,  |O        |
|Show     |NNP|O        |
|the      |DT |O        |
|number   |NN |O        |
+---------+---+---------+
only showing top 20 rows



In [11]:
training_data.select(F.explode(F.arrays_zip(training_data.token.result,
                                            training_data.label.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("token"),
                     F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)

+------------+-----+
|ground_truth|count|
+------------+-----+
|O           |11150|
|B-Cat       |2867 |
|B-Msr       |2840 |
|I-Msr       |2175 |
|I-Cat       |1265 |
|I-Goal      |105  |
|B-Goal      |99   |
|B-Tmln      |71   |
|I-Tmln      |20   |
+------------+-----+



In [12]:
# we can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)

glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [13]:
!mkdir ner_logs

A subdirectory or file ner_logs already exists.


In [14]:
nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(epochs)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_pipeline = Pipeline(stages=[
      glove_embeddings,
      nerTagger
 ])

### Fitting

In [15]:
%%time
#ner_pipeline = Pipeline(stages = [bert, nerTagger])
ner_model = ner_pipeline.fit(training_data)

# 1 epoch takes around 3 min with batch size=32
# if you get an error for incompatible TF graph, use 4.1 NerDL-Graph.ipynb notebook to create a graph (or see the bottom cell of this notebook)

Wall time: 13min 21s


In [16]:
import os

log_files = os.listdir("ner_logs")
log_files

['NerDLApproach_7150384162e7.log',
 'NerDLApproach_762ba26fb6e8.log',
 'NerDLApproach_839ecd20c5fb.log',
 'NerDLApproach_dbcc602d43a6.log',
 'NerDLApproach_dd6b5fad1fb4.log',
 'NerDLApproach_fcc75423c306.log',
 'NerDLApproach_fd2c2af7535c.log']

In [17]:
with open("ner_logs/"+log_files[0]) as log_file:
    print(log_file.read())

Name of the selected graph: ner-dl/blstm_10_128_128_120.pb
Training started - total epochs: 100 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 41 - training examples: 1210


Epoch 1/100 started, lr: 0.003, dataset size: 1210


Epoch 1/100 - 4.30s - loss: 297.1363 - batches: 40
Quality on validation dataset (20.0%), validation examples = 242
time to finish evaluation: 0.56s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-Tmln	 1	 1	 11	 0.5	 0.083333336	 0.14285715
I-Tmln	 0	 0	 2	 0.0	 0.0	 0.0
B-Goal	 0	 0	 4	 0.0	 0.0	 0.0
I-Msr	 105	 10	 114	 0.9130435	 0.47945204	 0.6287425
B-Msr	 253	 50	 44	 0.8349835	 0.8518519	 0.84333336
I-Goal	 0	 0	 4	 0.0	 0.0	 0.0
B-Cat	 284	 25	 8	 0.91909385	 0.9726027	 0.94509155
I-Cat	 124	 1	 22	 0.992	 0.84931505	 0.9151292
tp: 767 fp: 87 fn: 209 labels: 8
Macro-average	 prec: 0.5198901, rec: 0.4045694, f1: 0.45503697
Micro-average	 prec: 0.8981265, rec: 0.78586066, f1: 0.83825135


Epoch 2/100 started, lr: 0.0029850747, dataset size: 1210


Epoch 2/100 - 2.98

In [18]:
from sparknlp.training import CoNLL

test_data = CoNLL().readDataset(spark, test_CoNLL_file)

test_data = glove_embeddings.transform(test_data)

#test_data.show(f=False)

In [19]:
#test_data.select('sentence').take(20)

In [20]:
predictions = ner_model.transform(test_data)
predictions.show(3,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
predictions.select('token.result','label.result','ner.result').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                        |result                                                                                                                                      |result                                                                                                                                         |
+-------------------------------------------

### Test set evaluation

In [22]:
import pyspark.sql.functions as F

predictions.select(F.explode(F.arrays_zip(predictions.token.result,
                                          predictions.label.result,
                                          predictions.ner.result)).alias("cols")) \
                              .select(F.expr("cols['0']").alias("token"),
                                      F.expr("cols['1']").alias("ground_truth"),
                                      F.expr("cols['2']").alias("prediction")).show(truncate=False)

+---------+------------+----------+
|token    |ground_truth|prediction|
+---------+------------+----------+
|Show     |O           |O         |
|me       |O           |O         |
|the      |O           |O         |
|revenue  |B-Msr       |B-Msr     |
|across   |O           |O         |
|different|O           |O         |
|region   |B-Cat       |B-Cat     |
|,        |O           |O         |
|different|O           |O         |
|verticals|B-Cat       |B-Cat     |
|and      |O           |O         |
|across   |O           |O         |
|different|O           |O         |
|deparment|B-Cat       |B-Cat     |
|.        |O           |O         |
|Show     |O           |O         |
|me       |O           |O         |
|revenue  |B-Goal      |B-Msr     |
|for      |O           |O         |
|different|O           |O         |
+---------+------------+----------+
only showing top 20 rows



In [23]:
from sklearn.metrics import classification_report

preds_df = predictions.select(F.explode(F.arrays_zip(predictions.token.result,
                                                     predictions.label.result,
                                                     predictions.ner.result)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()

print (classification_report(preds_df['ground_truth'], preds_df['prediction']))

              precision    recall  f1-score   support

       B-Cat       0.85      0.88      0.86        25
      B-Goal       1.00      0.57      0.73         7
       B-Msr       0.64      0.78      0.70        32
      B-Tmln       0.65      0.73      0.69        15
       I-Cat       0.50      1.00      0.67         3
      I-Goal       1.00      1.00      1.00         3
       I-Msr       0.47      0.79      0.59        19
      I-Tmln       1.00      0.18      0.31        11
           O       0.96      0.90      0.93       227

    accuracy                           0.85       342
   macro avg       0.79      0.76      0.72       342
weighted avg       0.88      0.85      0.85       342



In [24]:
#!wget --help
#!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/utils/conll_eval.py
#!python -m wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/utils/conll_eval.py

In [25]:
import conll_eval

metrics = conll_eval.evaluate(preds_df['ground_truth'].values, preds_df['prediction'].values)

processed 342 tokens with 79 phrases; found: 86 phrases; correct: 50.
accuracy:  73.91%; (non-O)
accuracy:  84.80%; precision:  58.14%; recall:  63.29%; FB1:  60.61
              Cat: precision:  76.92%; recall:  80.00%; FB1:  78.43  26
             Goal: precision: 100.00%; recall:  57.14%; FB1:  72.73  4
              Msr: precision:  51.28%; recall:  62.50%; FB1:  56.34  39
             Tmln: precision:  35.29%; recall:  40.00%; FB1:  37.50  17


In [26]:
# micro, macro, avg
metrics[0]

(58.139534883720934, 63.29113924050633, 60.60606060606061)

In [27]:
import pandas as pd
pd.DataFrame(metrics[1], columns=['entity','precision','recall','f1','support'])

,entity,precision,recall,f1,support
0,Cat,76.923077,80.000000,78.431373,26
1,Goal,100.000000,57.142857,72.727273,4
2,Msr,51.282051,62.500000,56.338028,39
3,Tmln,35.294118,40.000000,37.500000,17


### Splitting Dataset Into Train and Test Set

In [28]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, train_CoNLL_file) #Training Data set

(training_data, test_data) = conll_data.randomSplit([0.7, 0.3], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 1949
Test Dataset Count: 918


In [29]:
training_data[training_data['text'] =="Show me number of promotions for all Geography"].show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Show me number of...|[{document, 0, 45...|[{document, 0, 45...|[{token, 0, 3, Sh...|[{pos, 0, 3, VB, ...|[{named_entity, 0...|
|Show me number of...|[{document, 0, 45...|[{document, 0, 45...|[{token, 0, 3, Sh...|[{pos, 0, 3, _, {...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [30]:
!mkdir ner_logs_best

A subdirectory or file ner_logs_best already exists.


In [31]:
glove_embeddings.transform(test_data).write.parquet('ner_dl_test.parquet')

In [32]:
%%time
#.setUseBestModel(True)\ - AttributeError: 'NerDLApproach' object has no attribute 'setUseBestModel'
nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(epochs)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('ner_dl_test.parquet')\
      .setOutputLogsPath('ner_logs_best') # if not set, logs will be written to ~/annotator_logs

ner_pipeline = Pipeline(stages=[
      glove_embeddings,
      nerTagger
 ])


ner_model = ner_pipeline.fit(training_data)

Wall time: 13min 2s


In [33]:
log_files = os.listdir("ner_logs_best/")
log_files

['NerDLApproach_7f585f0e2963.log',
 'NerDLApproach_83bfefe3ea81.log',
 'NerDLApproach_87838279f4d6.log',
 'NerDLApproach_9f47d4c947d7.log',
 'NerDLApproach_f37415f56e8c.log']

In [34]:
with open("ner_logs_best/"+log_files[0]) as log_file:
    print(log_file.read())

Name of the selected graph: ner-dl/blstm_10_100_128_120.pb
Training started - total epochs: 100 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 67 - training examples: 1563


Epoch 1/100 started, lr: 0.003, dataset size: 1563


Epoch 1/100 - 7.91s - loss: 225.38777 - batches: 51
Quality on validation dataset (20.0%), validation examples = 312
time to finish evaluation: 0.76s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-Tmln	 0	 0	 10	 0.0	 0.0	 0.0
I-Tmln	 0	 0	 3	 0.0	 0.0	 0.0
B-Goal	 0	 2	 15	 0.0	 0.0	 0.0
I-Msr	 281	 24	 17	 0.9213115	 0.942953	 0.93200666
B-Msr	 373	 30	 9	 0.9255583	 0.9764398	 0.9503185
I-Goal	 0	 0	 18	 0.0	 0.0	 0.0
B-Cat	 385	 15	 7	 0.9625	 0.98214287	 0.97222227
I-Cat	 171	 5	 1	 0.97159094	 0.99418604	 0.98275864
tp: 1210 fp: 76 fn: 80 labels: 8
Macro-average	 prec: 0.4726201, rec: 0.48696524, f1: 0.4796855
Micro-average	 prec: 0.940902, rec: 0.93798447, f1: 0.93944097
Quality on test dataset: 
time to finish evaluation: 0.72s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-T

In [35]:
test_data = glove_embeddings.transform(test_data)

predictions = ner_model.transform(test_data)

from sklearn.metrics import classification_report

preds_df = predictions.select(F.explode(F.arrays_zip(predictions.token.result,
                                                     predictions.label.result,
                                                     predictions.ner.result)).alias("cols")) \
                      .select(F.expr("cols['0']").alias("token"),
                              F.expr("cols['1']").alias("ground_truth"),
                              F.expr("cols['2']").alias("prediction")).toPandas()

print(classification_report(preds_df['ground_truth'], preds_df['prediction'], digits=4))

              precision    recall  f1-score   support

       B-Cat     0.9824    0.9814    0.9819       912
      B-Goal     0.8276    0.8000    0.8136        30
       B-Msr     0.9813    0.9803    0.9808       912
      B-Tmln     0.7143    0.5263    0.6061        19
       I-Cat     0.9860    0.9860    0.9860       429
      I-Goal     0.9231    0.6857    0.7869        35
       I-Msr     0.9569    0.9691    0.9630       711
      I-Tmln     0.5000    0.1667    0.2500         6
           O     0.9930    0.9964    0.9947      3578

    accuracy                         0.9839      6632
   macro avg     0.8739    0.7880    0.8181      6632
weighted avg     0.9833    0.9839    0.9834      6632



### Saving the trained model

In [36]:
ner_model.stages

[WORD_EMBEDDINGS_MODEL_48cffc8b9a76, NerDLModel_0c7f2b740869]

In [37]:
ner_model.stages[1].write().overwrite().save('Vwb_NER_glove_e5_b32')

In [38]:
#import pickle
#pickle.dump(ner_pipeline, open('Vwb_NER_glove_e5_b32.pkl','wb'))



### Prediction Pipeline

In [39]:
document = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

sentence = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentence')

token = Tokenizer()\
        .setInputCols(['sentence'])\
        .setOutputCol('token')

glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
        .setInputCols(["document", "token"])\
        .setOutputCol("embeddings")
  
# load trained model
loaded_ner_model = NerDLModel.load("Vwb_NER_glove_e5_b32")\
        .setInputCols(["sentence", "token", "embeddings"])\
        .setOutputCol("ner")

converter = NerConverter()\
        .setInputCols(["document", "token", "ner"])\
        .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(stages = [
        document,
        sentence,
        token,
        glove_embeddings,
        loaded_ner_model,
        converter
])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [40]:
empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)

In [41]:
#text = "Peter Compare my earned premium with unearned premium over the last five years."
#text = 'Show me the revenue across different region, different verticals and across different deparment.'
#text = 'show me expense report for last 3 years. Also show me my earned profit, unearned profit across claim type.'
#text = 'Show me the total expenses generated for last year. Also the gross sales vs the predicted sales for last 5 years. What is the revenue generated across different regions.'
#text = 'show me report on profit, loss basis on quarter. And show me my expense and revenue per country'
#text = 'Show me the total client across different age group.Also the total profit and revenue by age group. And the trend for average transcation for past 12 months'
#text = 'I would like to visualize profit achieved last year and also the loss incurred, What is the targeted revenue and profit for the next year.'
#text = 'share me report on my gross revenue, gross revenue by country and account.'
#text = 'Plot the total expense for the current month. Also the net profit vs the targeted profit.'
#text = 'I would like to visualize profit achieved last year and also the loss incurred, What is the targeted revenue and profit for the next year.'

#userStory = str(input())

#sample_data = spark.createDataFrame([[text]]).toDF("text")
sample_data = spark.createDataFrame([[userStory]]).toDF("text")

#sample_data.show(truncate=False)

NameError: name 'userStory' is not defined

In [ ]:
preds = prediction_model.transform(sample_data)

preds.select(F.explode(F.arrays_zip(preds.ner_span.result,preds.ner_span.metadata)).alias("entities")) \
      .select(F.expr("entities['0']").alias("chunk"),
              F.expr("entities['1'].entity").alias("entity")).show(truncate=False)

In [ ]:
preds.toDF

In [ ]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [ ]:
#text = "Peter Parker is a nice guy and lives in New York."
#text = 'show me expense report for last 3 years. Also show me my earned profit, unearned profit across claim type.'
#text = 'Show me the total expenses generated for last year. Also the gross sales vs the predicted sales for last 5 years. What is the revenue generated across different regions.'
#text = 'show me report on profit, loss basis on quarter. And show me my expense and revenue per country'
#text = 'Show me the total client across different age group. Also the total profit and revenue by age group. And the trend for average transcation for past 12 months'
text = 'I would like to visualize profit achieved last year and also the loss incurred, What is the targeted revenue and profit for the next year.'


result = light_model.annotate(text)

list(zip(result['token'], result['ner']))

In [ ]:
import pandas as pd

result = light_model.fullAnnotate(text)

ner_df= pd.DataFrame([(int(x.metadata['sentence']), x.result, x.begin, x.end, y.result) for x,y in zip(result[0]["token"], result[0]["ner"])], 
                      columns=['sent_id','token','start','end','ner'])
ner_df

In [ ]:
ner_df.to_csv('file1.csv')

### Highlight Entities

In [ ]:
! pip install -q spark-nlp_display

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

pipeline = PretrainedPipeline('recognize_entities_dl', lang='en')

In [ ]:
ann_text = pipeline.fullAnnotate('Show me the revenue across different region, different verticals and across different deparment.')[0]
ann_text.keys()

In [ ]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()
print ('Standard Output')
visualiser.display(ann_text, label_col='entities', document_col='document')

# Change color of an entity label
visualiser.set_label_colors({'Goal  ':'#008080', 'Cat':'#800080'})
visualiser.display(ann_text, label_col='entities')

# Set label filter
visualiser.display(ann_text, label_col='entities', document_col='document',
                   labels=['PER'])

print ('\nColor code for label: \n"Goal  ": {}\n"Category": {}' .format(visualiser.get_label_color('Goal  '),visualiser.get_label_color('Category')) )